In [2]:
import pandas as pd
import numpy as np
import pandas_profiling.profile_report as report

import re
import seaborn as sns
sns.set(style = 'white')
import matplotlib.pyplot as plt
from xgboost import plot_importance
%matplotlib inline
import matplotlib.ticker as mtick # For specifying the axes tick format 

import warnings
warnings.filterwarnings('ignore')

# Label Encoding
from sklearn.preprocessing import LabelEncoder

# modeling
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import model_selection
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, KFold

from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer, recall_score, log_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.metrics import plot_roc_curve
from sklearn.metrics import RocCurveDisplay

from scipy.special import softmax

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMClassifier

# 데이터준비

#### Data Prep

In [3]:
prc = pd.read_csv('./모델링시작.csv', encoding = 'euc-kr')
# prc.profile_report()

In [4]:
prc = prc.drop(['customer_status'], axis = 1)

#### Split data

In [24]:
X = prc.drop(['churn'], axis = 1)
y = prc['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3, stratify = y)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(4930, 30) (2113, 30)
(4930,) (2113,)


#### 표준화처리

In [25]:
prc.columns

Index(['gender', 'age', 'married', 'number_of_dependents',
       'number_of_referrals', 'tenure_in_months', 'offer', 'phone_service',
       'avg_monthly_long_distance_charges', 'multiple_lines',
       'internet_service', 'internet_type', 'avg_monthly_gb_download',
       'online_security', 'online_backup', 'device_protection_plan',
       'premium_tech_support', 'streaming_tv', 'streaming_movies',
       'streaming_music', 'unlimited_data', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charge', 'total_charges',
       'total_extra_data_charges', 'total_long_distance_charges',
       'total_revenue', 'tenure_in_years', 'churn'],
      dtype='object')

In [26]:
# 금액 칼럼 표준화
col = ['tenure_in_months', 'avg_monthly_long_distance_charges', 'monthly_charge', 'total_charges',
       'total_extra_data_charges', 'total_long_distance_charges',
       'total_revenue']

from sklearn.preprocessing import StandardScaler # 데이터 전처리: 표준점수화하기

ss = StandardScaler()
ss.fit(X_train[col])

X_train[col] = ss.transform(X_train[col])
X_test[col] = ss.transform(X_test[col]) 

# 모델 찾기

In [9]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(tree_method = 'hist'),
    'Support Vector Machine': SVC(probability=True),
    'Stacking': StackingClassifier(estimators=[('lr', LogisticRegression()),
                                               ('rf', RandomForestClassifier()), 
                                               ('gb', GradientBoostingClassifier())]),
    'Bagging': BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=10),
    'AdaBoost': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50),
    'Voting': VotingClassifier(estimators=[('lr', LogisticRegression()),
                                           ('rf', RandomForestClassifier()), 
                                           ('gb', GradientBoostingClassifier())]),
    'LightGBM': LGBMClassifier()
}

## Without Cross Validation

In [10]:
# Fit and evaluate models

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy = round(accuracy.mean(), 2)
    
    print(f'{model_name}:')
    print(f'Accuracy: {accuracy}')
    print(classification_report(y_test, y_pred))
    print()

# Voting
# Stacking
# GradientBoosting

Logistic Regression:
Accuracy: 0.81
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      1552
           1       0.64      0.67      0.66       561

    accuracy                           0.81      2113
   macro avg       0.76      0.77      0.76      2113
weighted avg       0.82      0.81      0.82      2113


Decision Tree:
Accuracy: 0.79
              precision    recall  f1-score   support

           0       0.87      0.84      0.85      1552
           1       0.59      0.64      0.61       561

    accuracy                           0.79      2113
   macro avg       0.73      0.74      0.73      2113
weighted avg       0.79      0.79      0.79      2113


Random Forest:
Accuracy: 0.82
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1552
           1       0.69      0.59      0.64       561

    accuracy                           0.82      2113
   macro avg       0.77      0

## Adding K-fold 

In [11]:
model_names = [] # 각 모델 이름 리스트
acc_results = [] # 검증세트 정확도 점수 저장할 리스트 생성
auc_results = []
pre_results = []
rec_results = []
f1_results = []

result_col = ["Algorithm", "Accuracy", "ROC AUC", "Precision", "Recall", "f1 Score"] # 데이터 프레임 칼럼 리스트
model_results = pd.DataFrame(columns = result_col) # 데이터 프레임 생성, 칼럼은 상위 칼럼 리스트

# Perform K-fold cross-validation
i=0
for model_name, model in models.items():
    model_names.append(model_name)
    kf = KFold(n_splits = 5)
    
    cv_acc_results = model_selection.cross_val_score(model, X_train, y_train, cv = kf, scoring = 'accuracy')
    cv_auc_results = model_selection.cross_val_score(model, X_train, y_train, cv = kf, scoring = 'roc_auc')
    cv_pre_results = model_selection.cross_val_score(model, X_train, y_train, cv = kf, scoring = 'precision')
    cv_rec_results = model_selection.cross_val_score(model, X_train, y_train, cv = kf, scoring = 'recall')
    cv_f1_results = model_selection.cross_val_score(model, X_train, y_train, cv = kf, scoring = 'f1')

    acc_results.append(cv_acc_results)
    auc_results.append(cv_auc_results)
    pre_results.append(cv_pre_results)
    rec_results.append(cv_rec_results)
    f1_results.append(cv_f1_results)
    
    model_results.loc[i] = [model_name,
                            np.mean(acc_results),
                            np.mean(auc_results),
                            np.mean(pre_results),
                            np.mean(rec_results),
                            np.mean(f1_results)]

    i+=1

model_results.sort_values(by = ['ROC AUC'], ascending=False)


,Algorithm,Accuracy,ROC AUC,Precision,Recall,f1 Score
0,Logistic Regression,0.839959,0.895493,0.708182,0.676300,0.691165
7,Bagging,0.829437,0.872018,0.732087,0.587782,0.636226
6,Stacking,0.827036,0.868865,0.728176,0.583543,0.629640
4,XGBoost,0.832211,0.864247,0.708894,0.639592,0.671473
5,Support Vector Machine,0.823124,0.862066,0.724390,0.570464,0.617547
8,AdaBoost,0.824701,0.855983,0.716993,0.587451,0.630833
3,Gradient Boosting,0.829817,0.855492,0.704448,0.637292,0.668332
2,Random Forest,0.821569,0.837169,0.686997,0.628150,0.655519
1,Decision Tree,0.811765,0.808522,0.649633,0.633933,0.643558
9,Voting,0.827647,NaN,0.720872,0.594201,0.637961


## LightGBM

In [12]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

pred_train_lgbm = lgbm.predict(X_train)
pred_test_lgbm = lgbm.predict(X_test)

acc_train_lgbm = accuracy_score(pred_train_lgbm, y_train)
acc_test_lgbm = accuracy_score(pred_test_lgbm, y_test)

print(f'Training accuracy {acc_train_lgbm: .3f}') 
print(f'Testing accuracy {acc_test_lgbm: .3f}')

Training accuracy  0.958
Testing accuracy  0.833


In [32]:
lgbm = LGBMClassifier()

def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_


param1 = [{'n_estimators': [100, 300, 500, 700],
           'learning_rate': [0.1, 0.2, 0.3],
           'max_depth': [1, 3, 5, 7, 9]}]
tuning_params(X_train, y_train, params = param1, model = lgbm)

Accuracy
Train score: 0.911156186612576
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
Best estimator:  LGBMClassifier(max_depth=3, n_estimators=300)
Best score:  0.8539553752535497

ROC_AUC
Train score: 0.9369717762839055
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Best estimator:  LGBMClassifier(max_depth=3)
Best score:  0.9117693876090017


(LGBMClassifier(max_depth=3, n_estimators=300), LGBMClassifier(max_depth=3))

Accuracy
Train score: 0.911156186612576
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
Best estimator:  LGBMClassifier(max_depth=3, n_estimators=300)
Best score:  0.8539553752535497

ROC_AUC
Train score: 0.9369717762839055
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Best estimator:  LGBMClassifier(max_depth=3)
Best score:  0.9117693876090017

(LGBMClassifier(max_depth=3, n_estimators=300), LGBMClassifier(max_depth=3))

In [36]:
good_lgbm = LGBMClassifier(learning_rate = 0.1, max_depth = 3, n_estimators = 100)
good_lgbm.fit(X_train, y_train)
pred_lgbm = good_lgbm.predict(X_test)

print('Results on test data: ')

acc_lgbm = accuracy_score(y_test, pred_lgbm)
pre_lgbm = precision_score(y_test, pred_lgbm)
rec_lgbm = recall_score(y_test, pred_lgbm)
auc_lgbm = roc_auc_score(y_test, pred_lgbm)

print(f'Test accuracy = {acc_lgbm: .4f}') 
print(f'Test precision = {pre_lgbm: .4f}') 
print(f'Test recall = {rec_lgbm: .4f}') 
print(f'Test auc = {auc_lgbm: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_lgbm))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_lgbm)))

Results on test data: 
Test accuracy =  0.8381
Test precision =  0.7017
Test recall =  0.6791
Test auc =  0.7874
Classification report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1552
           1       0.70      0.68      0.69       561

    accuracy                           0.84      2113
   macro avg       0.79      0.79      0.79      2113
weighted avg       0.84      0.84      0.84      2113

Confusion matrix (Rows actual, Columns predicted):
      0    1
0  1390  162
1   180  381


## Decision Tree

In [37]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

pred_train_dt = dt.predict(X_train)
pred_test_dt = dt.predict(X_test)

acc_train_dt = accuracy_score(pred_train_dt, y_train)
acc_test_dt = accuracy_score(pred_test_dt, y_test)

print(f'Training accuracy {acc_train_dt: .3f}') 
print(f'Testing accuracy {acc_test_dt: .3f}')

Training accuracy  1.000
Testing accuracy  0.783


In [38]:
dt = DecisionTreeClassifier()

def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_


param1 = {'criterion':['gini', 'entropy'], 'max_depth':[None,2,3,4,5,6]}
tuning_params(X_train, y_train, params = param1, model = dt)

Accuracy
Train score: 0.8446247464503043
Best params: {'criterion': 'gini', 'max_depth': 6}
Best estimator:  DecisionTreeClassifier(max_depth=6)
Best score:  0.8198782961460447

ROC_AUC
Train score: 0.9107539383009371
Best params: {'criterion': 'entropy', 'max_depth': 6}
Best estimator:  DecisionTreeClassifier(criterion='entropy', max_depth=6)
Best score:  0.8699115498330825


(DecisionTreeClassifier(max_depth=6),
 DecisionTreeClassifier(criterion='entropy', max_depth=6))

Accuracy
Train score: 0.8446247464503043
Best params: {'criterion': 'gini', 'max_depth': 6}
Best estimator:  DecisionTreeClassifier(max_depth=6)
Best score:  0.8198782961460447

ROC_AUC
Train score: 0.9107539383009371
Best params: {'criterion': 'entropy', 'max_depth': 6}
Best estimator:  DecisionTreeClassifier(criterion='entropy', max_depth=6)
Best score:  0.8699115498330825

(DecisionTreeClassifier(max_depth=6),
 DecisionTreeClassifier(criterion='entropy', max_depth=6))

In [39]:
good_dt = DecisionTreeClassifier(criterion='entropy', max_depth=6)
good_dt.fit(X_train, y_train)
pred_dt = good_dt.predict(X_test)

print('Results on test data: ')

acc_dt = accuracy_score(y_test, pred_dt)
pre_dt = precision_score(y_test, pred_dt)
rec_dt = recall_score(y_test, pred_dt)
auc_dt = roc_auc_score(y_test, pred_dt)

print(f'Test accuracy = {acc_dt: .4f}') 
print(f'Test precision = {pre_dt: .4f}') 
print(f'Test recall = {rec_dt: .4f}') 
print(f'Test auc = {auc_dt: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_dt))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_dt)))

Results on test data: 
Test accuracy =  0.8140
Test precision =  0.6556
Test recall =  0.6310
Test auc =  0.7556
Classification report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1552
           1       0.66      0.63      0.64       561

    accuracy                           0.81      2113
   macro avg       0.76      0.76      0.76      2113
weighted avg       0.81      0.81      0.81      2113

Confusion matrix (Rows actual, Columns predicted):
      0    1
0  1366  186
1   207  354


## Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)


pred_train_lr = lr.predict(X_train) # 훈련세트로 예측하기
pred_test_lr = lr.predict(X_test) # 테스트세트로 예측하기

acc_train_lr = accuracy_score(pred_train_lr, y_train) # 훈련 세트 예측과 훈련 세트 실제 타겟 간의 정확도 확인
acc_test_lr = accuracy_score(pred_test_lr, y_test) # 테스트 세트 예측과 테스트 세트 실제 타겟 간의 정확도 확인

print(f'Training accuracy {acc_train_lr: .3f}') 
print(f'Testing accuracy {acc_test_lr: .3f}')

### Tuning the parameters
Accuracy<br>
Train score: 0.845841784989858<br>
Best params: {'C': 10, 'max_iter': 500}<br>
Best estimator:  LogisticRegression(C=10, max_iter=500)<br>
Best score:  0.8434077079107505<br>
<br>
ROC_AUC<br>
Train score: 0.9024399819654608<br>
Best params: {'C': 10, 'max_iter': 900}<br>
Best estimator:  LogisticRegression(C=10, max_iter=900)<br>
Best score:  0.8982095605726684<br>
(LogisticRegression(C=10, max_iter=500),<br>
 LogisticRegression(C=10, max_iter=900))

In [ ]:
lr = LogisticRegression()

def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_


param1 = {'C':[10, 100, 1000], 'max_iter': [100, 300, 500, 700, 900, 1100]}
tuning_params(X_train, y_train, params = param1, model = lr)

### Evaluating

In [ ]:
good_lr = LogisticRegression(C = 10, max_iter = 900, random_state = 42)
good_lr.fit(X_train, y_train)
pred_lr = good_lr.predict(X_test)

print('Results on test data: ')

acc_lr = accuracy_score(y_test, pred_lr)
pre_lr = precision_score(y_test, pred_lr)
rec_lr = recall_score(y_test, pred_lr)
auc_lr = roc_auc_score(y_test, pred_lr)

print(f'Test accuracy = {acc_lr: .4f}') 
print(f'Test precision = {pre_lr: .4f}') 
print(f'Test recall = {rec_lr: .4f}') 
print(f'Test auc = {auc_lr: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_lr))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_lr)))

##### memo

In [ ]:
# 1안
# print('\nROC curve')
# plot_roc_curve(good_modelL, X_test, y_test)
# plt.show()


# 2안
# fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_lr)
# roc_auc = metrics.auc(fpr, tpr)
# display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Logistic Regression')
# display.plot() 

# 3안
# RocCurveDisplay(gb, X_test, y_test) 

In [ ]:
good_lr = LogisticRegression(C = 10, max_iter = 500, random_state = 42)
good_lr.fit(X_train, y_train)
pred_lr = good_lr.predict(X_test)

print('Results on test data: ')

acc_lr = accuracy_score(y_test, pred_lr)
pre_lr = precision_score(y_test, pred_lr)
rec_lr = recall_score(y_test, pred_lr)
auc_lr = roc_auc_score(y_test, pred_lr)

print(f'Test accuracy = {acc_lr: .4f}') 
print(f'Test precision = {pre_lr: .4f}') 
print(f'Test recall = {rec_lr: .4f}') 
print(f'Test auc = {auc_lr: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_lr))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_lr)))


## Bagging

In [ ]:
# 순정 테스트
bag = BaggingClassifier(base_estimator=RandomForestClassifier(random_state=42), 
                        random_state = 42, n_jobs = -1)

bag.fit(X_train, y_train)
pred_train_bag = bag.predict(X_train)
pred_test_bag = bag.predict(X_test)

acc_train_bag = accuracy_score(pred_train_bag, y_train) # 훈련 세트 예측과 훈련 세트 실제 타겟 간의 정확도 확인
acc_test_bag = accuracy_score(pred_test_bag, y_test) # 테스트 세트 예측과 테스트 세트 실제 타겟 간의 정확도 확인

print(f'Training accuracy {acc_train_rf: .3f}') 
print(f'Testing accuracy {acc_test_rf: .3f}')

In [ ]:
bag = BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25, max_leaf_nodes=24, n_estimators=75, random_state=42), 
                        n_estimators=10, oob_score = True, bootstrap = True, random_state = 42, n_jobs = -1)

bag.fit(X_train, y_train)
pred_train_bag = bag.predict(X_train)
pred_test_bag = bag.predict(X_test)

acc_train_bag = accuracy_score(pred_train_bag, y_train) # 훈련 세트 예측과 훈련 세트 실제 타겟 간의 정확도 확인
acc_test_bag = accuracy_score(pred_test_bag, y_test) # 테스트 세트 예측과 테스트 세트 실제 타겟 간의 정확도 확인

print(f'Training accuracy {acc_train_rf: .3f}') 
print(f'Testing accuracy {acc_test_rf: .3f}')

### Tuning the parameters
Accuracy <br>
Train score: 0.8614604462474645 <br>
Best params: {'max_samples': 1000, 'n_estimators': 200} <br>
Best estimator:  BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25,
                                                        max_leaf_nodes=24,
                                                        n_estimators=75,
                                                        random_state=42),
                  max_samples=1000, n_estimators=200, n_jobs=-1, oob_score=True,
                  random_state=42) <br>
Best score:  0.8446247464503042 <br>
 <br>
ROC_AUC <br>
Train score: 0.9104615947058158 <br>
Best params: {'max_samples': 1000, 'n_estimators': 100} <br>
Best estimator:  BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25,
                                                        max_leaf_nodes=24,
                                                        n_estimators=75,
                                                        random_state=42),
                  max_samples=1000, n_estimators=100, n_jobs=-1, oob_score=True,
                  random_state=42) <br>
Best score:  0.8911536533705606 <br>
(BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25,
                                                         max_leaf_nodes=24,
                                                         n_estimators=75,
                                                         random_state=42),
                   max_samples=1000, n_estimators=200, n_jobs=-1, oob_score=True,
                   random_state=42), <br>
 BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25,
                                                         max_leaf_nodes=24,
                                                         n_estimators=75,
                                                         random_state=42),
                   max_samples=1000, n_estimators=100, n_jobs=-1, oob_score=True,
                   random_state=42)) <br>

In [ ]:
def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_

# (max_features=25, max_leaf_nodes=24, n_estimators=75, random_state=42)
# bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, n_jobs=-1)
# (DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True)
param1 = [{'n_estimators': [100, 200, 300, 10, 50],
          'max_samples': [10, 100, 1000],
          }]
tuning_params(X_train, y_train, params = param1, model = bag)


### Evaluating

In [ ]:
good_bag = BaggingClassifier(base_estimator=RandomForestClassifier(max_features=25, max_leaf_nodes=24, n_estimators=75, random_state=42), 
                             max_samples=1000, n_estimators=100, n_jobs=-1, oob_score=True, random_state=42)
good_bag.fit(X_train, y_train)
pred_bag = good_bag.predict(X_test)

print('Results on test data: ')

acc_bag = accuracy_score(y_test, pred_bag)
pre_bag = precision_score(y_test, pred_bag)
rec_bag = recall_score(y_test, pred_bag)
auc_bag = roc_auc_score(y_test, pred_bag)

print(f'Test accuracy = {acc_bag: .4f}') 
print(f'Test precision = {pre_bag: .4f}') 
print(f'Test recall = {rec_bag: .4f}') 
print(f'Test auc = {auc_bag: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_bag))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_bag)))

## Random Forest

In [ ]:
rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)


pred_train_rf = rf.predict(X_train) # 훈련세트로 예측하기
pred_test_rf = rf.predict(X_test) # 테스트세트로 예측하기

acc_train_rf = accuracy_score(pred_train_rf, y_train) # 훈련 세트 예측과 훈련 세트 실제 타겟 간의 정확도 확인
acc_test_rf = accuracy_score(pred_test_rf, y_test) # 테스트 세트 예측과 테스트 세트 실제 타겟 간의 정확도 확인

print(f'Training accuracy {acc_train_rf: .3f}') 
print(f'Testing accuracy {acc_test_rf: .3f}')

### Tuning the parameters

In [ ]:
def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_

param1 = [{'n_estimators': [25, 50, 75],
           'max_features': [15, 20, 25],
           'max_leaf_nodes': [8, 16, 24]}]
tuning_params(X_train, y_train, params = param1, model = rf)


### Evaluating

In [ ]:
good_rf = RandomForestClassifier(max_features=25, max_leaf_nodes=24, n_estimators=75, random_state=42)

good_rf.fit(X_train, y_train)
pred_rf = good_rf.predict(X_test)

print('Results on test data: ')

acc_rf = accuracy_score(y_test, pred_rf)
pre_rf = precision_score(y_test, pred_rf)
rec_rf = recall_score(y_test, pred_rf)
auc_rf = roc_auc_score(y_test, pred_rf)

print(f'Test accuracy = {acc_rf: .4f}') 
print(f'Test precision = {pre_rf: .4f}') 
print(f'Test recall = {rec_rf: .4f}') 
print(f'Test auc = {auc_rf: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_rf))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_rf)))

## Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, X_train, y_train, return_train_score = True, n_jobs = -1) # 교차검증 + 정확도 확인

print("Train Score:", np.mean(scores['train_score']))
print("Test Score:", np.mean(scores['test_score']))

gb.fit(X_train, y_train)

pred_train_gb = gb.predict(X_train)
pred_test_gb = gb.predict(X_test)

acc_train_gb = accuracy_score(pred_train_gb, y_train)
acc_test_gb = accuracy_score(pred_test_gb, y_test)

print(f'Training accuracy {acc_train_gb: .3f}') 
print(f'Testing accuracy {acc_test_gb: .3f}')


### Tuning the parameter

In [ ]:
def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_

param1 = [{'n_estimators': [80, 100, 150, 200, 500],
         'learning_rate': [0.1, 0.2, 0.3],
         'max_depth': [1, 3, 5, 7, 9]}]
tuning_params(X_train, y_train, params = param1, model = gb)


##### need?

### Evaluating

In [ ]:
good_gb = GradientBoostingClassifier(n_estimators = 300, learning_rate = 0.1, max_depth = 5, random_state = 42)
                            # learning_rate GB의 중요한 매개변수. 이전 트리의 오차를 얼마나 강하게 보정할 것인지를 제어
                            # n_estimators 트리 더 많이 추가. 모델의 복잡도가 커지고 훈련 세트에서의 실수를 바로 잡을 기회가 더 많아짐

good_gb.fit(X_train, y_train)
pred_gb = good_gb.predict(X_test)

print('Results on test data: ')

acc_gb = accuracy_score(y_test, pred_gb)
pre_gb = precision_score(y_test, pred_gb)
rec_gb = recall_score(y_test, pred_gb)
auc_gb = roc_auc_score(y_test, pred_gb)

print(f'Test accuracy = {acc_gb: .4f}') 
print(f'Test precision = {pre_gb: .4f}') 
print(f'Test recall = {rec_gb: .4f}') 
print(f'Test auc = {auc_gb: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_gb))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_gb)))

In [ ]:
print(gb.feature_importances_)
print(good_gb.feature_importances_)

## XGBoost

In [ ]:
xgb =XGBClassifier(tree_method = 'hist', random_state = 42) # (n_estimators=700, learning_rate=0.1, max_depth=3)
xgb.fit(X_train,y_train)

pred_train_xgb = xgb.predict(X_train)
pred_test_xgb = xgb.predict(X_test)

acc_train_xgb = accuracy_score(pred_train_xgb, y_train)
acc_test_xgb = accuracy_score(pred_test_xgb, y_test)

print(f'Training accuracy {acc_train_xgb: .3f}') 
print(f'Testing accuracy {acc_test_xgb: .3f}')

### Tuning the parameters

In [ ]:
def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_

param1 = [{'n_estimators': [100, 300, 500, 700],
           'learning_rate': [0.1, 0.2, 0.3],
           'max_depth': [1, 3, 5, 7, 9]}]

tuning_params(X_train, y_train, params = param1, model = xgb)


### Evaluating

In [ ]:
good_xgb = XGBClassifier(tree_method = 'hist', learning_rate = 0.1, max_depth = 3, n_estimators = 500, random_state = 42)

evals = [(X_test, y_test)]
good_xgb.fit(X_train, y_train, 
             early_stopping_rounds = 100, eval_metric="logloss", eval_set = evals, verbose=True)
pred_xgb = good_xgb.predict(X_test)


print('Results on test data: ')

acc_xgb = accuracy_score(y_test, pred_xgb)
pre_xgb = precision_score(y_test, pred_xgb)
rec_xgb = recall_score(y_test, pred_xgb)
auc_xgb = roc_auc_score(y_test, pred_xgb)

print(f'Test accuracy = {acc_xgb: .4f}') 
print(f'Test precision = {pre_xgb: .4f}') 
print(f'Test recall = {rec_xgb: .4f}') 
print(f'Test auc = {auc_xgb: .4f}')

print("Classification report:")
print(classification_report(y_test, pred_xgb))

print("Confusion matrix (Rows actual, Columns predicted):")
print(pd.DataFrame(confusion_matrix(y_test, pred_xgb)))

### Feature Importance

In [ ]:
fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(good_xgb, ax=ax)

## Support Vector Machine

In [ ]:
svmc = SVC(random_state = 42, probability=True)
svmc.fit(X_train, y_train)

pred_train_svmc = svmc.predict(X_train)
pred_test_svmc = svmc.predict(X_test)

acc_train_svmc = accuracy_score(pred_train_svmc, y_train)
acc_test_svmc = accuracy_score(pred_test_svmc, y_test)

print(f'Training accuracy {acc_train_svmc: .3f}') 
print(f'Testing accuracy {acc_test_svmc: .3f}')

### Tuning the parameters

In [ ]:
def tuning_params(X, y, params, model):
    gs_acc = GridSearchCV(model, params, scoring = 'accuracy', cv = 5, n_jobs = -1)
    gs_auc = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5, n_jobs = -1)
    
    gs_acc.fit(X, y)
    gs_auc.fit(X, y)
    
    print('Accuracy')
    print('Train score:', gs_acc.score(X_train, y_train))
    print('Best params:', gs_acc.best_params_)
    print("Best estimator: ", gs_acc.best_estimator_)
    print("Best score: ", gs_acc.best_score_)
    
    print('\nROC_AUC')
    print('Train score:', gs_auc.score(X_train, y_train))
    print('Best params:', gs_auc.best_params_)
    print("Best estimator: ", gs_auc.best_estimator_)
    print("Best score: ", gs_auc.best_score_)
    
    return gs_acc.best_estimator_, gs_auc.best_estimator_

param1 = [{'kernel':['linear', 'rbf'], 'gamma':[1, 0.1, 0, 0.3, 0.5, 0.7], 'C':[1, 10, 100]}]

tuning_params(X_train, y_train, params = param1, model = svmc)

### Evaluating

## 최적 모델 찾기

## 고객 이탈을 예측

In [ ]:
# 고객 데이터 임의 설정
gender = 1
age = 25
married = 0
number_of_dependents = 0
number_of_referrals = 2
tenure_in_months = 7
offer = 2
phone_service = 0
avg_monthly_long_distance_charges = 0
multiple_lines = 0
internet_service = 1
internet_type = 2
avg_monthly_gb_download = 45
online_security = 1
online_backup = 0
device_protection_plan = 1
premium_tech_support = 0
streaming_tv = 1
streaming_movies = 0
streaming_music = 1
unlimited_data = 0
contract = 0
paperless_billing = 1
payment_method = 2
monthly_charge = 65
total_charges = 
total_revenue = Total Charges - Total Refurnds + Total Extra Data Charges + Total Lond Distance Charges